# init spark session

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("test_spark")
    .config("spark.driver.memory", "1g")
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.warehouse.dir", "hdfs://namenode:9000/user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    # .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    # .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.executor.cores", 3)
    .config("spark.executor.memory", "6g")
    .config("spark.executor.instances", 2)
    .enableHiveSupport()
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 06:21:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

# test hive table

In [3]:
spark.sql(f"""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
spark.sql(f"""
create database staging
location 'hdfs://namenode:9000/user/hive/warehouse/staging'
""").show()

++
||
++
++



In [5]:
spark.sql(f"""
show databases
""").show()

+---------+
|namespace|
+---------+
|  default|
|  staging|
+---------+



In [6]:
spark.sql(f"""
use staging
""").show()

++
||
++
++



In [8]:
spark.sql(f"""
show tables
""").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [9]:
spark.sql(f"""
create table if not exists staging.test_hive (key int, value string)
stored as orc
location 'hdfs://namenode:9000/user/hive/warehouse/staging/test_hive'
""").show()

24/07/23 06:28:55 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [10]:
spark.sql(f"""
show tables
""").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  staging|test_hive|      false|
+---------+---------+-----------+



In [11]:
spark.sql(f"""
insert into staging.test_hive
values (1, "2")
""").show()

++
||
++
++



In [13]:
spark.sql(f"""
select * from staging.test_hive
""").show()

+---+-----+
|key|value|
+---+-----+
|  1|    2|
+---+-----+



# test iceberg table